In [ ]:
import readers
import functions
import pandas as pd
import numpy as np
import csv
import matplotlib.pyplot as plt

# Parameter lists for every person (modify if necessary)

In [ ]:
#Parameter lists
distance = ['back','4back','2teacher','4teacher']                      #Adrián
renewalshour = [2,3,4,8]                                               #Vighnesh
roomtemp = [19,23,27]                                                  #Ronghao
exposuretime = ['60s','120s','240s','480s']                            #Yibing
masktype = ['wall','cotton','N95']                                     #Malo
maskNumber = ['one','all']                                             #Malo
ncoughpeople = ['back','back+teach','front+teach','sides+teach','all'] #Sócrates
caseAdri = 'distance'
caseVig = 'ventilation'
caseRong = 'temp'
caseYib = 'student'
caseMal = 'maskType'
caseMal2 = 'maskNumber'
caseSoc = 'npeople'
#For mixed build:
paramlist = [ncoughpeople,renewalshour,roomtemp,exposuretime,masktype,maskNumber,distance]
caseslist = [caseSoc,caseVig,caseRong,caseYib,caseMal,caseMal2,caseAdri]
overalldict = dict(zip(caseslist,paramlist))

### Build mixed dictionaries based on list mixedlist

In [ ]:
mixedlist = ['back','wall','sides+teach',23,8]
timesmixdict = dict.fromkeys(mixedlist)
massmixdict = dict.fromkeys(mixedlist)
totalsmixdict = dict.fromkeys(mixedlist)
zonesmixdict = dict.fromkeys(mixedlist)
back = False
for elem in mixedlist:
    if elem == 'back' and back == False:
        back = True
    elif elem == 'back' and back == True:
        continue
    for casename,plist in overalldict.items():
        print(casename,plist,elem)
        if elem in plist:
            zn,td,md,totd = functions.build(casename,[elem])
            functions.matchallnames(td,md,zn,[elem])
            timesmixdict[elem] = td[elem]
            massmixdict[elem] = md[elem]
            totalsmixdict[elem] = totd[elem]
            zonesmixdict[elem] = zn[elem]
            break
        else:
            print(elem,' not found in mixeddict')

##### Load data with your casename and your parameter

In [ ]:
zones,timesdict,massdict,totalsdict = functions.build(caseSoc,ncoughpeople)
functions.matchallnames(timesdict,massdict,zones,ncoughpeople)

### Calculate % of total particles and % of total mass for each fate. New columns are 'Particles %' and 'Mass %'

In [ ]:
for key,item in timesdict.items():
    #we cannot use item as the dataframe to which apply the operation because item is a copy of the actual dataframe inside the dict
    timesdict[key] = functions.percentage(timesdict[key],'Number','Particles',totalsdict[key])
    timesdict[key].sort_index(axis=0,level='flowtime',inplace=True)
for key,item in massdict.items():
    massdict[key] = functions.percentage(massdict[key],'Final','Mass',totalsdict[key])
    massdict[key].sort_index(axis=0,level='flowtime',inplace=True)
for key,item in totalsdict.items():
    totalsdict[key].sort_index(axis=0,level='flowtime',inplace=True)
#For single parameter element:
#timesdict['2m'] = percentage(timesdict['2m'],'Number','Particles',totalsdict['2m'])
#massdict['2m'] = percentage(massdict['2m'],'Final','Mass',totalsdict['2m'])

## Same (percentages columns) for mixed dicts

In [ ]:
for key,item in timesmixdict.items():
    #we cannot use item as the dataframe to which apply the operation because item is a copy of the actual dataframe inside the dict
    timesmixdict[key] = functions.percentage(timesmixdict[key],'Number','Particles',totalsmixdict[key])
    timesmixdict[key].sort_index(axis=0,level='flowtime',inplace=True)
for key,item in massmixdict.items():
    massmixdict[key] = functions.percentage(massmixdict[key],'Final','Mass',totalsmixdict[key])
    massmixdict[key].sort_index(axis=0,level='flowtime',inplace=True)
for key,item in totalsmixdict.items():
    totalsmixdict[key].sort_index(axis=0,level='flowtime',inplace=True)
#For single parameter element:
#timesdict['2m'] = percentage(timesdict['2m'],'Number','Particles',totalsdict['2m'])
#massdict['2m'] = percentage(massdict['2m'],'Final','Mass',totalsdict['2m'])

### Export the state at a chosen instant to LaTeX table. 
Default number format is 2 decimals. If you need to change it do it inside functions.py or tell me

In [ ]:
toprintcolumns = ['Fate','Number','Particles %']
instant = 60
#pass whatever instant and list of columns to instant_to_latex
functions.instant_to_latex(timesdict,toprintcolumns,instant)

### Drop a given list of matches (default is floor surfs, mouths, Escaped)
Returns the filtered dictionary

In [ ]:
filterstrings = ['mouth','Trapped '+r'floor\d{1,2}','Escaped','roof','students_floor4','ceiling']
newtimesdict = functions.dropfates(timesdict,filterstrings) #drops all mouths, Escaped and floor surfaces (keeps non-floor zones labeled with floor in their name)
newmassdict = functions.dropfates(massdict,filterstrings)

#### Same for mixed dict

In [ ]:
filterstrings = ['mouth','Trapped '+r'floor\d{1,2}','Escaped','roof','students_floor4','ceiling']
newtimesmixdict = functions.dropfates(timesmixdict,filterstrings) #drops all mouths, Escaped and floor surfaces (keeps non-floor zones labeled with floor in their name)
newmassmixdict = functions.dropfates(massmixdict,filterstrings)

### Plot against parameter

In [ ]:
functions.plotagainstparam(timesdict,'Particles %','Trapped',40,'Trapped')
functions.plotagainstparam(timesdict,'Particles %','Fluid',40,'in Fluid')
plt.legend()
plt.title('Trapped particles in %')

## BAR PLOT EXAMPLE
### --------------------------------------

In [ ]:
fates = ['desk','student','inFluid','Escaped']
x = np.arange(len(timesdict.keys()))
instant = 60
fix,ax,handles,bars = functions.barplot(timesdict,'Particles %',instant,fates)
ax.set_xticks(x)
ax.set_xticklabels(timesdict.keys())
ax.set_ylabel('Particles %')
ax.set_title('Fate of particles at t=' + str(instant))
ax.legend()
plt.savefig('testbar.png',dpi=250)

#### Same for mixed dict

In [ ]:
fates = ['desk','student','inFluid','mask']
x = np.arange(len(timesmixdict.keys()))
instant = 60
fix,ax,handles,bars = functions.barplot(timesmixdict,'Particles %',instant,fates)
ax.set_xticks(x)
ax.set_xticklabels(timesmixdict.keys())
ax.set_ylabel('Particles %')
ax.set_title('Fate of particles at t=' + str(instant))
ax.legend()
plt.savefig('testbar.png',dpi=250)

## Time Evolution plot
## -----------------------------

In [ ]:
fates = ['Fluid','desk','teacher','student']
functions.plotagainsttime(timesdict,fates)

#### Same for mixed

In [ ]:
fates = ['Fluid','Trapped']
functions.plotagainsttime(newtimesmixdict,fates)

### Other things

In [ ]:
fates = timesdict['back+teach'].loc[60,'Fate'].append(timesdict['sides+teach'].loc[60,'Fate'],ignore_index=True).drop_duplicates()

In [ ]:
timesdict['front+teach'].index

In [ ]:
column_across_param(timesdict,'Number',60)

# EXAMPLES ON HOW TO ACCESS THE DATA:

In [ ]:
timesdict['sides+teach'].loc[timesdict['sides+teach'].Fate.str.contains(fr"desk",regex=True)]

In [ ]:
massdict['2m'].loc[60]

In [ ]:
totalsdict

In [ ]:
massdict['4m']['Mass %']

In [ ]:
timesdict['2m'].loc[0.6,'Particles %']

### Print several documentations (out of date though)

In [ ]:
print('READ_RESULTS INSTRUCTIONS')
print('----------------------------------------------')
print(read_results.__doc__)
print('-----------------------------------------------')
print('')
print('PERCENTAGE INSTRUCTIONS')
print('----------------------------------------------')
print(percentage.__doc__)
print('-----------------------------------------------')
print('')
print('PLOT AGAINST PARAM INSTRUCTIONS')
print('----------------------------------------------')
print(plotagainstparam.__doc__)
print('MATCHNAMES INSTRUCTIONS')
print('----------------------------------------------')
print(readers.matchnames.__doc__)